# Avaliação do melhor modelo:

### Imports

In [5]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import (
    roc_curve, auc, precision_recall_curve, average_precision_score,
    classification_report, confusion_matrix, precision_score, recall_score, f1_score
)

from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image

## Curvas de aprendizado

### Curvas de acurácia

In [ ]:
# Carrega os históricos salvos
all_acc = np.load('all_acc.npy')
all_val_acc = np.load('all_val_acc.npy')

mean_acc = np.mean(all_acc, axis=0)
std_acc = np.std(all_acc, axis=0)
mean_val_acc = np.mean(all_val_acc, axis=0)
std_val_acc = np.std(all_val_acc, axis=0)
epochs = np.arange(1, len(mean_acc)+1)

melhor_idx = np.argmax([val_acc[-1] for val_acc in all_val_acc])
melhor_acc = all_acc[melhor_idx]
melhor_val_acc = all_val_acc[melhor_idx]

plt.figure(figsize=(8,5))

plt.plot(epochs, mean_acc, label='Treino (média 20x)', color='#1f77b4', linestyle='-', marker='o', linewidth=2, markersize=5)
plt.fill_between(epochs, mean_acc-std_acc, mean_acc+std_acc, alpha=0.10, color='#1f77b4')

plt.plot(epochs, mean_val_acc, label='Validação (média 20x)', color='#d62728', linestyle='-', marker='s', linewidth=2, markersize=5)
plt.fill_between(epochs, mean_val_acc-std_val_acc, mean_val_acc+std_val_acc, alpha=0.10, color='#d62728')

plt.plot(epochs, melhor_acc, label='Treino (melhor)', color='#1f77b4', linestyle='--', marker='^', linewidth=2, markersize=6)

plt.plot(epochs, melhor_val_acc, label='Validação (melhor)', color='#d62728', linestyle=':', marker='x', linewidth=2, markersize=7)

plt.title('Acurácia por Época', fontsize=15)
plt.xlabel('Época', fontsize=13)
plt.ylabel('Acurácia', fontsize=13)
plt.legend(fontsize=11, loc='lower right', frameon=True)
plt.grid(alpha=0.25)
plt.tight_layout()
plt.show()

### Curvas de Loss

In [ ]:
# Carrega os históricos salvos
all_loss = np.load('all_loss.npy')
all_val_loss = np.load('all_val_loss.npy')

mean_loss = np.mean(all_loss, axis=0)
std_loss = np.std(all_loss, axis=0)
mean_val_loss = np.mean(all_val_loss, axis=0)
std_val_loss = np.std(all_val_loss, axis=0)
epochs = np.arange(1, len(mean_loss)+1)

melhor_idx = np.argmin([val_loss[-1] for val_loss in all_val_loss])
melhor_loss = all_loss[melhor_idx]
melhor_val_loss = all_val_loss[melhor_idx]

plt.figure(figsize=(8,5))

plt.plot(epochs, mean_loss, label='Treino (média 20x)', color='#2ca02c', linestyle='-', marker='o', linewidth=2, markersize=5)
plt.fill_between(epochs, mean_loss-std_loss, mean_loss+std_loss, alpha=0.10, color='#2ca02c')

plt.plot(epochs, mean_val_loss, label='Validação (média 20x)', color='#ff7f0e', linestyle='-', marker='s', linewidth=2, markersize=5)
plt.fill_between(epochs, mean_val_loss-std_val_loss, mean_val_loss+std_val_loss, alpha=0.10, color='#ff7f0e')

plt.plot(epochs, melhor_loss, label='Treino (melhor)', color='#2ca02c', linestyle='--', marker='^', linewidth=2, markersize=6)

plt.plot(epochs, melhor_val_loss, label='Validação (melhor)', color='#ff7f0e', linestyle=':', marker='x', linewidth=2, markersize=7)

plt.title('Loss por Época', fontsize=15)
plt.xlabel('Época', fontsize=13)
plt.ylabel('Loss', fontsize=13)
plt.legend(fontsize=11, loc='upper right', frameon=True)
plt.grid(alpha=0.25)
plt.tight_layout()
plt.show()

## Gráfico de Curva AUC-ROC

In [ ]:
# Carregar melhor modelo salvo
model = load_model('melhor_modelo.h5')


test_gen = ImageDataGenerator(rescale=1./255)
test_data = test_gen.flow_from_directory(
    'dataset2_aumentado/test', target_size=(224,224), batch_size=16, class_mode='binary', shuffle=False
)

y_true = test_data.classes
y_pred_prob = model.predict(test_data).flatten()

fpr, tpr, thresholds = roc_curve(y_true, y_pred_prob)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(7,6))
plt.plot(fpr, tpr, color='#d62728', lw=2, label=f'ROC curve (AUC = {roc_auc:.3f})')
plt.plot([0, 1], [0, 1], color='gray', lw=1, linestyle='--')
plt.xlabel('Taxa de Falsos Positivos (1 - Especificidade)')
plt.ylabel('Taxa de Verdadeiros Positivos (Sensibilidade)')
plt.title('Curva ROC do Melhor Modelo')
plt.legend(loc='lower right')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## Gráfico de Precision-Recall

In [ ]:
precision, recall, thresholds = precision_recall_curve(y_true, y_pred_prob)
ap = average_precision_score(y_true, y_pred_prob)

plt.figure(figsize=(7,6))
plt.plot(recall, precision, color='black', lw=2, label=f'AP = {ap:.3f}')
plt.xlabel('Recall')
plt.ylabel('Precisão')
plt.title('Curva Precision-Recall do Melhor Modelo')
plt.legend(loc='lower left')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## Matriz de Confusão melhor modelo

In [ ]:
# Threshold padrão 0.5
y_pred = (y_pred_prob > 0.5).astype(int)

print("Classification Report (melhor modelo):")
print(classification_report(y_true, y_pred, target_names=['nao_poluido', 'poluido']))

print("Matriz de Confusão:")
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['nao_poluido', 'poluido'], yticklabels=['nao_poluido', 'poluido'])
plt.xlabel('Predito')
plt.ylabel('Verdadeiro')
plt.title('Matriz de Confusão - Melhor Modelo')
plt.tight_layout()
plt.show()

## Exibição em casos reais de erros e acertos do modelo:

In [ ]:
# Caminhos das imagens do teste
filepaths = [test_data.filepaths[i] for i in range(len(test_data.filepaths))]

acertos = np.where(y_true == y_pred)[0]
erros = np.where(y_true != y_pred)[0]

np.random.shuffle(acertos)
np.random.shuffle(erros)

def plot_examples(indices, title, n=4):
    plt.figure(figsize=(12,3))
    for i, idx in enumerate(indices[:n]):
        img = image.load_img(filepaths[idx], target_size=(128,128))
        plt.subplot(1, n, i+1)
        plt.imshow(img)
        plt.axis('off')
        plt.title(f"V: {y_true[idx]}, P: {y_pred[idx]}")
    plt.suptitle(title)
    plt.tight_layout()
    plt.show()

plot_examples(acertos, "Exemplos de Acertos", n=4)

plot_examples(erros, "Exemplos de Erros", n=4)

## Matriz de confusão e análise de métricas para diferentes thresholds

In [ ]:
thresholds_to_test = [0.3, 0.5, 0.7, 0.9]

precision_list = []
recall_list = []
f1_list = []

print("Análise para diferentes thresholds:\n")

for thresh in thresholds_to_test:
    y_pred_thresh = (y_pred_prob > thresh).astype(int)
    
    precision = precision_score(y_true, y_pred_thresh)
    recall = recall_score(y_true, y_pred_thresh)
    f1 = f1_score(y_true, y_pred_thresh)

    precision_list.append(precision)
    recall_list.append(recall)
    f1_list.append(f1)

    cm = confusion_matrix(y_true, y_pred_thresh)
    
    print(f"Threshold = {thresh:.2f}")
    print(f"Precision: {precision:.3f} | Recall: {recall:.3f} | F1-Score: {f1:.3f}")
    print("Matriz de Confusão:")
    print(cm)
    print("-"*50)

    plt.figure(figsize=(4.5,4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['não poluído', 'poluído'], 
                yticklabels=['não poluído', 'poluído'])
    plt.title(f'Matriz de Confusão (Threshold={thresh:.2f})')
    plt.xlabel('Predito')
    plt.ylabel('Verdadeiro')
    plt.tight_layout()
    plt.show()

# Plot das métricas em função do threshold
plt.figure(figsize=(8,5))
plt.plot(thresholds_to_test, precision_list, marker='o', label='Precision')
plt.plot(thresholds_to_test, recall_list, marker='o', label='Recall')
plt.plot(thresholds_to_test, f1_list, marker='o', label='F1-Score')
plt.xlabel('Threshold')
plt.ylabel('Valor da métrica')
plt.title('Variação das Métricas com o Threshold')
plt.grid(True, linestyle='--', alpha=0.5)
plt.legend()
plt.tight_layout()
plt.show()

# Matriz de Confusão média das 20 execuções

In [ ]:
conf_matrices = np.load('all_conf_matrices.npy')  # shape: (20, 2, 2)

mean_cm = np.mean(conf_matrices, axis=0)

class_labels = ['nao_poluido', 'poluido']

plt.figure(figsize=(6, 5))
sns.heatmap(mean_cm, annot=True, fmt='.2f', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predito')
plt.ylabel('Verdadeiro')
plt.title('Matriz de Confusão Média (20 execuções)')
plt.show()